In [ ]:

#import pakcgages yang diperlukan
from bs4 import BeautifulSoup as sp
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import re
import numpy as np

opsi=webdriver.ChromeOptions()
servis=Service('chromedriver.exe')
driver = webdriver.Chrome(service=servis, options=opsi)

SMM_Link=SMM['link']

data_fix_2=[]        
for link in SMM_Link:
    url=driver.get(link)
    time.sleep(5)
    content=driver.page_source
    beautify=sp(content,'html.parser')
    try:
        harga=beautify.find('div',{'data-testid':"lblPDPDetailProductPrice"}).text
        harga=int(harga[2:].replace('.',''))
    except:
        harga=np.nan
    
    try:
        terjual=beautify.find('p',{'data-testid':"lblPDPDetailProductSoldCounter"}).text
        terjual=re.search(r'\d+',terjual)
        terjual=terjual.group()
        print(terjual)
    except:
        terjual=np.nan
        print("nan")
    
    try:    
        bintang=beautify.find('span',{'data-testid':"lblPDPDetailProductRatingNumber"}).text
        bintang=float(bintang)
    except:
        bintang=np.nan

    try:
        jumlah_perating=beautify.find('span',{'data-testid':"lblPDPDetailProductRatingCounter"}).text
        jumlah_perating=re.search(r'\d+',jumlah_perating)
        jumlah_perating=jumlah_perating.group()
        jumlah_perating=int(jumlah_perating)
    except:
        jumlah_perating=np.nan

    try:
        deskripsi=beautify.find('div',{'data-testid':"lblPDPDescriptionProduk"}).text
    except:
        deskripsi=''
    
    try:
        images=beautify.find('img',{'data-testid':"PDPMainImage"})
        images=str(images)
        image = re.findall(r'src="(.*?)".*?"(.*?)',images)
        image = image.group(1)
    except:
        image=''
    data_fix_2.append([harga,terjual,bintang,jumlah_perating,deskripsi,image])
data_fix_2=pd.DataFrame(data_fix_2,columns=['harga','terjual','bintang','jumlah_perating','deskripsi','image'])
data_fix_2.to_csv('data_desc.csv',index=False)


#Concating Process
import pandas as pd
import matplotlib as plt
import numpy as np

nama_link=pd.read_csv("benar.csv",index_col=False)
lainya=pd.read_csv("data_desc.csv",index_col=False)
#Melakukan penggabungan dengan concat
Scrape_Tokped=pd.concat([nama_link, lainya], axis=1)
#MElakukan perubahan urutan column sesuai keiningan
Scrape_Tokped=Scrape_Tokped[['product','harga','terjual','bintang','jumlah_perating','deskripsi','link']]

#Melakukan grouping untuk mendapat informasi mengenai 
def category(product):
    if 'Baju' in product:
        return 'Baju'
    elif 'Celana' in product:
        return 'Celana'
    elif 'Kemeja' in product:
        return 'Kemeja'
    elif 'Kebaya' in product:
        return 'Kebaya'
    elif 'Rok' in product:
        return 'Rok'
    elif 'Bandana'in product:
        return 'Bandana'
    elif 'Tali' in product:
        return 'Tali'
    elif 'Tas' in product:
        return 'Tas'
    else:
        return 'Lainnya'

data=pd.read_csv('Scrape_Tokped.csv')
df=data.copy()
df['category']=df['product'].apply(category)
df.to_csv('Scrape_to_PowerBI')
